# BeautifulSoup 크롤링
만든이 : 오건우


# 1. 네이버 정치 일반 뉴스 크롤링

In [40]:
#네이버 정치 일반 크롤링

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

url = "http://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269"

html = urlopen(url)
soup = BeautifulSoup(html,"html.parser",from_encoding='utf-8')

In [41]:
articles = soup.findAll("dt")


In [42]:
#기사: <li> - <dl>

ex_dl = articles[0].find("a")["href"]
print(ex_dl)

https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=025&aid=0002922107


In [43]:
url_list = []

for link in articles:
    url_list.append(link.find("a")["href"])
    
url_list

['https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=025&aid=0002922107',
 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=025&aid=0002922107',
 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010952114',
 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010952114',
 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010952113',
 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010952113',
 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010952112',
 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010952112',
 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010952111',
 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010952111',


In [44]:
url_list2 = list(set(url_list)) ##중복 제거


# len(url_list2) 20개 -> 맞나 확인

In [45]:

html = urlopen(url_list[13])
soup = BeautifulSoup(html,"html.parser",from_encoding='utf-8')

content = soup.find("div",{"id":"articleBodyContents"})
title = soup.find("h3",{"id":"articleTitle"}).find(text=True)

#content.find("br")#.find(text=True) # 'ResultSet' object has no attribute 'findAll'
#title
#content.findAll("br")[2].findAll(text=True)

#[]안에있는거 제거, 불필요한 것 제거

In [46]:
import re
p = re.compile('\<.*?\>') #괄호안에 있는거 다 지우겠다.

test = str(content.text)

real_text = re.sub(p, "", test)



In [47]:
#정규표현식으로 영어, 특수문자 제거해줍니다.
# -*- coding: utf-8 -*-
import re
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자

test = str(content.find("br"))
result = hangul.sub('', test) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
print (result)

#반복되는 필요없는 

In [48]:
#뉴스 페이지 넘어가면서 크롤링해보기
url = "http://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=269&sid1=100&date=20180320&page="
texts = []
titles = []
url_list = []
hangul = re.compile('\<.*?\>') # 괄호 안에 있는 것

for i in range(2):
    page_url = url + str(i)
    html = urlopen(page_url)
    soup = BeautifulSoup(html,"html.parser",from_encoding='utf-8')
    articles = soup.findAll("dt")
   
    
    for link in articles:
        url_list.append(link.find("a")["href"])
        
url_list2 = list(set(url_list)) ##중복 제거
        
for url in url_list2:
    html_article = urlopen(url)
    soup_article = BeautifulSoup(html_article,"html.parser",from_encoding='utf-8')

    content = soup_article.find("div",{"id":"articleBodyContents"})
    title = soup_article.find("h3",{"id":"articleTitle"}).find(text=True)

    content.find("br")
            
    real_text = hangul.sub('', str(content.find("br")))
            
    titles.append(title)
    texts.append(real_text)

In [30]:
len(texts)

20

In [462]:
import pandas as pd
from pandas import DataFrame
import numpy as np

data = [titles, texts]

newData = np.transpose(data)

df = DataFrame(newData, columns = ["기사 제목","내용"])


기사 제목  \
0          설훈 국회 농해수위원장, 2018년도 농협중앙회 정기대위원회 격려사   
1                              답변하는 김영주 고용노동부 장관   
2                                답변하는 김현미 국토부 장관   
3                            평화당·정의당 공동교섭을 위한 인사   
4                                    답변하는 김현미 장관   
5                                이재정 교육감 '재선 출마'   
6           [서울포토] 주한 베트남 유학생들에게 청와대 안내하는 김정숙 여사   
7                                    질의하는 김현아 의원   
8                 여야 개헌 공방 격화…대통령 개헌안 놓고 난상토론 예고   
9                              답변하는 김영주 고용노동부 장관   
10                            [포토] 활짝 웃는 장병완-노회찬   
11                    이명박 전 대통령, 영장실질심사 불출석하기로 해   
12                         재선 출마 질문에 답하는 이재정 교육감   
13                    국토위원회 1차 전체회의 주재하는 조정식 위원장   
14                            재선 출마 내비치는 이재정 교육감   
15                         재선 출마 질문에 답하는 이재정 교육감   
16                            물 마시는 김영주 고용노동부 장관   
17                                 발언하는 노회찬 원내대표   
18                            [포토] 밝게 웃는 장병완-노회찬   
19                                   답변하는 김현미 장관   
20                          평화당·정의당 공동교섭단체 구성 회동   
21                             답변하는 김영주 고용노동부 장관   
22                             청와대, 대통령 개헌안 첫 발표   
23           [서울포토] 김정숙 여사, 주한 베트남 유학생들에게 청와대 안내   
24                                   질의하는 김현아 의원   
25                  MB, 영장심사에 출석 안 해…"이미 충분히 밝혀"   
26                           국토위 회의 주재하는 조정식 위원장   
27                           국토위 회의 주재하는 조정식 위원장   
28            MB 영장심사 22일 오전 10시30분…MB 측"출석 안한다"   
29                      논의하는 국토위 조정식 위원장과 이원욱 간사   
..                                           ...   
50                           국토위 회의 주재하는 조정식 위원장   
51                박홍근 "한국당, 3월 임시국회 소집으로 상임위 방해"   
52                                인사말하는 장병완 원내대표   
53                            물 마시는 김영주 고용노동부 장관   
54                  [포토] 평화당-정의당, 공동원내교섭단체 구성 합의   
55                            [포토] 발언하는 노회찬 원내대표   
56              [현장연결] 靑, 대통령 개헌안 '전문ㆍ기본권 분야' 공개   
57                       이명박 영장실질심사 22일 결정, 구속될까   
58        MB 구속영장 실질심사, 22일 오전 10시 30분…박범석 판사 심리   
59                                        함께합시다!   
60                            [포토] 발언하는 장병완 원내대표   
61                            [포토] 발언하는 장병완 원내대표   
62                          꿈의학교 결과 발표하는 이재정 교육감   
63           [서울포토] 김정숙 여사, 주한 베트남 유학생 초청 오찬 간담회   
64             [개헌안 공개] 대통령 개헌안 발의에 ‘與 vs 4野’ 구도   
65  [단독] 한국당 중진들 “인재영입 안되면 홍준표 대표가 서울시장 선거 나가라”    
66                      의사봉 두드리는 조정식 국토교통위원회 위원장   
67                            안건 상정하는 조정식 국토위위원장   
68                                   답변하는 김현미 장관   
69                         재선 출마 질문에 답하는 이재정 교육감   
70                논의하는 국토위 조정식 위원장과 자유한국당 박덕흠 간사   
71                        MB 운명 가를 박범석 부장판사는 누구?   
72                          평화당·정의당 공동교섭단체 구성 회동   
73                               답변하는 김현미 국토부 장관   
74                             [포토] 손 잡는 평화당-정의당   
75       정봉주 “민주당, 파울했다고 선수자격 정지하겠다는 것과 다를 바 없어”   
76                                        함께합시다!   
77                      재선 출마 관련 질문에 답하는 이재정 교육감   
78        MB, 구속영장 심사 불출석 의사..."검찰에 입장 충분히 밝혔다"    
79                          평화당·정의당 공동교섭단체 구성 회동   

                                                   내용  
0   서울뉴시스배훈식 기자  일 오전 서울 서대문구 농협중앙회 대강당에서 열린 년도 정기...  
1       서울연합뉴스 김현태 기자  일 오전 국회에서 열린 환경노동위원회 전체회의에서...  
2   서울뉴시스홍효식 기자  일 오전 서울 여의도 국회 본청 국토교통위원회 회의실에서 열...  
3       서울연합뉴스 백승렬 기자  민주평화당 장병완 원내대표와 정의당 노회찬 원내대...  
4       서울연합뉴스 김현태 기자  일 오전 국회에서 열린 국토교통위원회 전체회의에서...  
5   수원뉴시스이정선 기자  이재정 경기교육감이 일 오전 수원 경기교육청 브리핑룸에서 열...  
6      문재인 대통령 부인 김정숙 여사가 베트남 순방을 앞두고 지난 일 청와대에서 주...  
7   서울뉴시스홍효식 기자  일 오전 서울 여의도 국회 본청 국토교통위원회 회의실에서 열...  
8   한국 헌법 전문 먹칠누더기 헌법서울연합뉴스 김경희 설승은 기자  문재인 대통령의 정...  
9       서울연합뉴스 김현태 기자  일 오전 국회에서 열린 환경노동위원회 전체회의에서...  
10  장병완 민주평화당 원내대표와 노회찬 정의당 원내대표가 일 국회에서 만나 인사를 나누...  
11  이 전 대통령 비서실은 일 입장 자료를 통해 구속 전 피의자 심문영장실질심사에 출석...  
12  수원뉴시스이정선 기자  이재정 경기교육감이 일 오전 수원 경기교육청 브리핑룸에서 열...  
13  서울뉴시스홍효식 기자  일 오전 서울 여의도 국회 본청

# 2. 네이버 영화 평점 크롤링

In [368]:
url = "https://movie.naver.com/movie/point/af/list.nhn?page=1"

html = urlopen(url)
soup = BeautifulSoup(html,"html.parser",from_encoding='utf-8')

reviews = soup.findAll("tr")


[<tr>
 <th>번호</th>
 <th colspan="2">평점</th>
 <th>140자평</th>
 <th class="al"><span class="th_m1">글쓴이·날짜</span></th>
 </tr>, <tr>
 <td class="ac num">13908853</td>
 <td><div class="fr point_type_n"><div class="mask" style="width:70.0%"></div></div></td>
 <td class="point">7</td>
 <td class="title">
 <a class="movie" href="?st=mcode&amp;sword=150371&amp;target=after">토니 에드만</a>
 <br>암튼 평레기들은 대사 적고 호흡 긴 영화 좋아하고, 이 영화처럼 배경음악 없는 영화 또한 선호한다 
 			
 			
 			
 				
 				
 				
 				<a class="report" href="javascript:report('pass****', 'wOE0D8RFXIgWG7/hb6WbtekRPnEu9CQhELlO3ihfdpY=', '암튼 평레기들은 대사 적고 호흡 긴 영화 좋아하고, 이 영화처럼 배경음악 없는 영화 또한 선호한다', '13908853', 'point_after');" style="color:#8F8F8F">신고</a>
 </br></td>
 <td class="num"><a class="author" href="javascript:find_list('nickname','13908853','after');">pass****</a><br>18.03.19</br></td>
 </tr>, <tr>
 <td class="ac num">13908852</td>
 <td><div class="fr point_type_n"><div class="mask" style="width:90.0%"></div></div></td>
 <td class="point">9</td>


In [371]:
#평점
points = reviews[1].find("td",{"class":"point"})
print(points)

point = []

for p in reviews:
    point.append(p.find("td",{"class":"point"}))


point

#숫자만 남기면됨


<td class="point">7</td>


[None,
 <td class="point">7</td>,
 <td class="point">9</td>,
 <td class="point">10</td>,
 <td class="point">8</td>,
 <td class="point">3</td>,
 <td class="point">10</td>,
 <td class="point">10</td>,
 <td class="point">1</td>,
 <td class="point">10</td>,
 <td class="point">1</td>]

In [372]:
reviews[0].find("a",{"class":"movie"}).find(text=True)

#None 때문에 텍스트=트루가 안되잖아??????

AttributeError: 'NoneType' object has no attribute 'find'

In [374]:
#제목


for title in reviews[1:]:
    print(title.find("a",{"class":"movie"}).find(text=True))


#None이 0에 나오니까 1번째부터 가줍니다


토니 에드만
팬텀 스레드
완벽한 거짓말
플로리다 프로젝트
나미야 잡화점의 기적
월요일이 사라졌다
벤지
치즈인더트랩
지금 만나러 갑니다
120BPM


In [387]:

reviews[1].find("a",{"class":"report"})

#reviews[1].find("a",{"class":"report"}).find(text=True)


#reviews[1].find("a",{"class":"report"})["href"]



<a class="report" href="javascript:report('pass****', 'wOE0D8RFXIgWG7/hb6WbtekRPnEu9CQhELlO3ihfdpY=', '암튼 평레기들은 대사 적고 호흡 긴 영화 좋아하고, 이 영화처럼 배경음악 없는 영화 또한 선호한다', '13908853', 'point_after');" style="color:#8F8F8F">신고</a>

In [393]:
# 리뷰


contents = []

for content in reviews[1:]:
    contents.append(content.find("a",{"class":"report"})["href"])
    
contents[1]



"javascript:report('5083****', '0NWx8ca5VZhQdmNO66iaQ6uSGns685IvsyAkCOst9H8=', '한참전에 일본판보고 그 여운이 깊게 남아서 볼까말까 고민하다가 오늘 보고 왔는데 보고 오길 잘 한 것 같아요 비현실적인 부분도 있었지만 뭐 영화니까 가능한 거라 생각.. 내용을 알고 봤는데도 감정이입되면서 웃기도하고 울기도하고 뒤로 갈수록 코끝 찡~', '13908779', 'point_after');"

In [395]:

#정규표현식으로 영어, 특수문자 제거해줍니다.
# -*- coding: utf-8 -*-
import re
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자

result = hangul.sub('', contents[1]) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
print (result)



  한참전에 일본판보고 그 여운이 깊게 남아서 볼까말까 고민하다가 오늘 보고 왔는데 보고 오길 잘 한 것 같아요 비현실적인 부분도 있었지만 뭐 영화니까 가능한 거라 생각 내용을 알고 봤는데도 감정이입되면서 웃기도하고 울기도하고 뒤로 갈수록 코끝 찡  


In [400]:
#페이지 넘어가면서 자동화하기 50페이지까지 긁어보기

url_list = []

for i in range(51):
    url = "https://movie.naver.com/movie/point/af/list.nhn?&page="+str(i)
    url_list.append(url)
    
url_list[1:51]
points = []
titles = []
contents = []

for url in url_list[1:10]:
    #평점

    html = urlopen(url)
    soup = BeautifulSoup(html,"html.parser",from_encoding='utf-8')

    reviews = soup.findAll("tr")
    
    for item in reviews[1:]:
        titles.append(item.find("a",{"class":"movie"}).find(text=True))
        points.append(item.find("td",{"class":"point"}).find(text=True))
        contents.append(item.find("a",{"class":"report"})["href"])
        
        

['리틀 포레스트',
 '해리 포터와 마법사의 돌',
 '쓰리 빌보드',
 '사라진 밤',
 '지금 만나러 갑니다',
 '쓰리 빌보드',
 '블랙 팬서',
 '치즈인더트랩',
 '지금 만나러 갑니다',
 '120BPM',
 '1급기밀',
 '리틀 포레스트',
 '블랙 팬서',
 '쓰리 빌보드',
 '리틀 포레스트',
 '셰이프 오브 워터: 사랑의 모양',
 '일진',
 '스케치',
 '리틀 포레스트',
 '셰이프 오브 워터: 사랑의 모양',
 '스타워즈: 라스트 제다이',
 '셰이프 오브 워터: 사랑의 모양',
 '지금 만나러 갑니다',
 '아이, 토냐',
 '120BPM',
 '토니 에드만',
 '팬텀 스레드',
 '완벽한 거짓말',
 '플로리다 프로젝트',
 '나미야 잡화점의 기적',
 '월요일이 사라졌다',
 '벤지',
 '치즈인더트랩',
 '지금 만나러 갑니다',
 '120BPM',
 '쓰리 빌보드',
 '맨체스터 바이 더 씨',
 '사라진 밤',
 '은밀하게 위대하게',
 '지금 만나러 갑니다',
 '지금 만나러 갑니다',
 '쓰리 빌보드',
 '120BPM',
 '120BPM',
 '지금 만나러 갑니다',
 '사라진 밤',
 '세 번째 살인',
 '지금 만나러 갑니다',
 '사라진 밤',
 '지금 만나러 갑니다',
 '120BPM',
 '셰이프 오브 워터: 사랑의 모양',
 '의뢰인',
 '설리: 허드슨강의 기적',
 '골든슬럼버',
 '치즈인더트랩',
 '120BPM',
 '세컨핸드 라이온스',
 '120BPM',
 '지금 만나러 갑니다',
 '지금 만나러 갑니다',
 '120BPM',
 '리틀 포레스트',
 '왓 위민 원트',
 '120BPM',
 '썸머 워즈',
 '특별수사: 사형수의 편지',
 '지금 만나러 갑니다',
 '블랙 팬서',
 '120BPM',
 '골든슬럼버',
 '보통사람',
 '120BPM',
 '쓰리 빌보드',
 '120BPM',
 '지금 만나러 갑니다',
 '엄마의 공책',
 '그때 그사람들',
 '페르디난드',
 '사라진 

In [401]:
import pandas as pd
from pandas import DataFrame
import numpy as np

data = [titles, points, contents]

newData = np.transpose(data)

df = DataFrame(newData, columns = ["제목","평점","리뷰"])


제목  평점                                                 리뷰
0             리틀 포레스트   7  javascript:report('jjae****', 'DX5NONRipf5FtAf...
1       해리 포터와 마법사의 돌  10  javascript:report('euju****', 'AU1xJqi5TM4rgq7...
2              쓰리 빌보드  10  javascript:report('dcha****', 'JmV34dmoDAIAbDO...
3               사라진 밤   8  javascript:report('ikmu****', '7SGtrGLbBin8t+i...
4          지금 만나러 갑니다  10  javascript:report('jang****', 'Q87jmfkVjiEVWFk...
5              쓰리 빌보드  10  javascript:report('park****', 'Rx+1b0wRcYy2/xe...
6               블랙 팬서   8  javascript:report('jhsn****', 'QGlVr0FsZJLPFO1...
7              치즈인더트랩   1  javascript:report('dlqh****', 'Hof4t1qrQJ6ECaQ...
8          지금 만나러 갑니다  10  javascript:report('bebe****', 'e86a6UEsn1q3Z9G...
9              120BPM   1  javascript:report('lie5****', 'A/mGKztbkuSNdhu...
10               1급기밀  10  javascript:report('kts2****', '5u1ZnsZ7jgNlXfr...
11            리틀 포레스트  10  javascript:report('tndu****', 'r1gdj2Uwtlpt19U...
12              블랙 팬서   8  javascript:report('ju00****', 'NI7L5X8vZ6A0Mbk...
13             쓰리 빌보드   9  javascript:report('hitc****', 'NNJuizUhyvO8OYv...
14            리틀 포레스트  10  javascript:report('rlaa****', 'kevLjw5BeyYuScu...
15  셰이프 오브 워터: 사랑의 모양   8  javascript:report('hitc****', '0oAnMYXBDWSeH0V...
16                 일진  10  javascript:report('ehda****', 'nPahblNDtWx0qM4...
17                스케치   7  javascript:report('lps4****', '+dTenwP/xcVn8p0...
18            리틀 포레스트  10  javascript:report('dnal****', 'PAASvWdFrB1fQSt...
19  셰이프 오브 워터: 사랑의 모양   4  javascript:report('manz****', 'OF7ijPWmByqzFiU...
20      스타워즈: 라스트 제다이   6  javascript:report('dowo****', 'wPFEiAMY5VYj1hT...
21  셰이프 오브 워터: 사랑의 모양   8  javascript:report('ming****', 'D6vnacT0mO6BD7R...
22         지금 만나러 갑니다   9  javascript:report('lhh0****', '64UDCyKRpeHSPoW...
23             아이, 토냐   6  javascript:report('ming****', 'fAPz5R32qJkEF+J...
24             120BPM   1  javascript:report('pmjj****', '8jBX3VX90VJ1FLJ...
25             토니 에드만   7  javascript:report('pass****', 'M2bRdo2tdjtxJPV...
26             팬텀 스레드   9  javascript:report('ming****', 'TylimUHXQ8qeMB1...
27            완벽한 거짓말  10  javascript:report('halo****', 'fRP8/vLu/Reh3yf...
28          플로리다 프로젝트   8  javascript:report('rimr****', 'uQ89g8Lpu7peoj5...
29        나미야 잡화점의 기적   3  javascript:report('mpmp****', 'VjtW54oieZCqcuM...
..                ...  ..                                                ...
60         지금 만나러 갑니다  10  javascript:report('trau****', 'GGCqZCXeWQaS2Rr...
61             120BPM   1  javascript:report('hsi8****', 'TPDRvoA1ItttbrZ...
62            리틀 포레스트   7  javascript:report('a110****', '3DJi9Ha8UEJJvrY...
63            왓 위민 원트   9  javascript:report('bbie****', 'TS2J/w70KG988ug...
64             120BPM   1  javascript:report('wodn****', 'ZbmLrgBmOtF9fO4...
65              썸머 워즈  10  javascript:report('qwee****', 'u5iXfGfvA3LyRP8...
66      특별수사: 사형수의 편지   8  javascript:report('fubu****', 'mpW/uEbQqXxSVql...
67         지금 만나러 갑니다  10  javascript:report('toho****', '080KKzof74buUVe...
68              블랙 팬서   8  javascript:report('zzag****', 'd3FAlZwzpKdCUiE...
69             120BPM   1  javascript:report('qw88****', 'pPF7wHbhbQwzguS...
70              골든슬럼버   4  javascript:report('dani****', 'nS5uRCXSZ1YUFH7...
71               보통사람   7  javascript:report('sute****', 'jU3n9L+QGYHJa97...
72             120BPM   1  javascript:report('kcw7****', 'Au2finaWyU1aLdE...
73             쓰리 빌보드  10  javascript:report('bgh3****', 'FZflQmBdlXibGNJ...
74             120BPM   1  javascript:report('hlch****', 'Xru4lzgqodJjkaa...
75         지금 만나러 갑니다   4  javascript:report('hong****', 'q0n4vE55wCxis72...
76             엄마의 공책  10  javascript:report('fldr****', 'f95gVp3QVewraB+...
77            그때 그사람들   9  javascript:report('2eme****', 'z3uEFrhAiUwnIBj...
78              페르디난드   8  javascript:report('deff****', 'IIKZiEz6114W7Og...
79              사라진 밤  10  javascript:report('wiku****', 'ICFkOsWVHypi8Xy...
80      스타워즈: 라

In [403]:

import re
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') 

contents_hangel = []
for content in contents:
    contents_hangel.append(hangul.sub('', content))

    
data = [titles, points, contents_hangel]

newData = np.transpose(data)

df = DataFrame(newData, columns = ["제목","평점","리뷰"])



제목  평점                                                 리뷰
0             리틀 포레스트   7                                청춘들의 성장통 자연스러운 향기  
1       해리 포터와 마법사의 돌  10    와진짜 나의 최에영화다내가 이 해리포터를 어렸를때 안좋은 기억으로 알고있었는데다...
2              쓰리 빌보드  10    보는 내내 한시도 눈을 뗄 수 없었던 스토리 전개였다 미친 연기력까지 최고였음 ...
3               사라진 밤   8          폐쇄된지역에서 이야기가 전개되다보니 지루한감이있었지만 떡밥회수는 잘해줌  
4          지금 만나러 갑니다  10    일본판을 봤던 터라 기대를 안했는데 알고봐도 미친 연기력들 덕분에 감정이 우는거...
5              쓰리 빌보드  10    영화 중반부터 몰입도 쩐다영화가 끝나가면서 아 이건 다시 봐야겠다는 생각이 들었...
6               블랙 팬서   8                                   볼만한 영화 입니다 보세요  
7              치즈인더트랩   1    끝나고 이게 뭐지라고 생각한 영화싱크만 똑같고 다다른 느낌 개연성 없고백인하 백...
8          지금 만나러 갑니다  10                              최고 말이필요없음 부부라면 꼭보세요  
9              120BPM   1                                    노잼 노잼 노잼 슈퍼노잼  
10               1급기밀  10                                    미디어가 세상을 바로잡다  
11            리틀 포레스트  10                             좋았다 요리를 배우고 싶게 만든 영화  
12              블랙 팬서   8                                    좋았습니다ㅏ 재밋게봤어요  
13             쓰리 빌보드   9    처음에 엔딩을 보고 좀 급하게 끝난 것 같다는 생각을 했는데 극장을 나오면서 정...
14            리틀 포레스트  10                                 힐링영화 귀농하고 싶게 만든다  
15  셰이프 오브 워터: 사랑의 모양   8    음악이 너무 좋았다 진부할법한 주제를 전혀 새로운 이미지로 그려낸 점이 대단히 ...
16                 일진  10                            암세포들이 암걸려 죽었습니다 감사합니다  
17                스케치   7    영상과 음악은 좋다 다만 스토리가 지루했음 분짜리 영화면 차라리 나았을지도 그리...
18            리틀 포레스트  10                                    쌀밥같은 영화같아 좋네요  
19  셰이프 오브 워터: 사랑의 모양   4                        전반적으로 개연성이 떨어져서 와닿지 않았습니다  
20      스타워즈: 라스트 제다이   6    베스트 평점 보고 먹고 있던거 뿜은 적은 처음이다 우리 나라 한개 사단이 입에 ...
21  셰이프 오브 워터: 사랑의 모양   8           사랑의 여러모양 그 여러모양은 비난받아서도 안되고 비난 해서도 안된다  
22         지금 만나러 갑니다   9                        영상이 예쁘고 괜찮네요 손예진 연기 넘 잘해요  
23             아이, 토냐   6    블랙코미디의 넘치는 에너지와 현란한 편집이 초반엔 가장 큰 강점이었지만 후반으로...
24             120BPM   1                                  똥꼬박멸 싫어하는것도 권리다  
25             토니 에드만   7    암튼 평레기들은 대사 적고 호흡 긴 영화 좋아하고 이 영화처럼 배경음악 없는 영...
26             팬텀 스레드   9                     올리풀려버리는 사랑 그러다가도 묘하게 직조되는 사랑  
27            완벽한 거짓말  10    매순간순간 주인공의 선택에 나도 같이 선택을한다 그 선택들이 모여 주인공이 파국...
28          플로리다 프로젝트   8                                신선하고 먹먹해지는  영화였어요  
29        나미야 잡화점의 기적   3    감정을 극대화시키기 위해 눈물 클로즈업이 많았고 영화를 보는 것보다 원작을 읽는...
..                ...  ..                                                ...
60         지금 만나러 갑니다  10                                너무 감동적이고 아름다운 이야기  
61             120BPM   1                                    돈고춘 역겹읍니다 ㄹㅇ루  
62            리틀 포레스트   7                                  난 이런 담백한 영화가 좋다  
63            왓 위민 원트   9                                      다시봐도 너무 재밌다  
64             120BPM   1                                     피던지는건 개에바아니냐  
65              썸머 워즈  10                                 여름 분위기 일본분위기 물씬남  
66      특별수사: 사형수의 편지   8                          권선징악은 돈과 관계없다라는건 영화속에서나  
67         지금 만나러 갑니다  10                             최근 영화중 등 한참을 울다가 웃다가  
68              블랙 팬서   8                         생각보다그냥 저냥이었던듯 시간 죽이기는 좋음  
69             120BPM   1                                  더러움 그자체 영화 토나오네  
70              골든슬럼버   4    리메이크를 왜 한거지  원작의 정서도 이해하지 못하고영화 자체의 만듦새도 일본판...
71               보통사람   7                    그래도 지금 이 시대는 적어도 그때 보단 나은 삶이다  
72             120BPM   1                       게이레즈페미니스트 등은 질병이다 치료가 필요하다  
73             쓰리 빌보드  10    입체적인 캐릭터와 환상적 각본의 조화 영화 보는 내내 시간가는 줄 모르고 몰입했...
74             120BPM   1                                          디러웡웡웡ㅇㅇ  
75         지금 만나러 갑니다   4        일본영화만큼 감성이 짙지않아서 실망을 많이 했다 그래도 찍느라 고생하셨어요  
76             엄마의 공책  10    잔잔한 우리의 삶을 볼수있었습니다 부모님과 함께 보고 왔는데요 오랜만에 극장나들...
77            그때 그사람들   9                             자네같은 사람은 야전에 있어야 하는데  
78              페르디난드   8    소가 주인공이라고 해서 솔직히 기대하지 않았는데 너무 귀엽고 조연으로 나온 캐릭...
79              사라진 밤  10        근래에 본 영화중 가장 재미있게 봤습니다 배우들 케미도 좋고 전체적으로 굿  
80      스타워즈: 라

# 3. 네이버 실시간 검색어 데이터



In [17]:
# 1. html
import requests
from bs4 import BeautifulSoup

response = requests.get('http://www.naver.com/')

bs = BeautifulSoup(response.text, 'html.parser')
type(bs)


bs4.BeautifulSoup

실시간 검색어 키워드

In [18]:
elements = bs.findAll("li",{"class":"ah_item"})


In [19]:
elements[0].find("span",{"class":"ah_k"})

<span class="ah_k">로또867회당첨번호</span>

In [20]:
for keyword in elements:
    print(keyword.find("span",{"class":"ah_k"}).find(text=True))
    

로또867회당첨번호
원더풀 고스트
선우선
보좌관 시즌2
호텔 델루나
이수민
안미나
원피스 893화 애니
델루나
인기가요
미구하라
트와이스 미나
제니하우스 염색약
우하람
홍윤화
있지
아스달 연대기 방송시간
정세진 아나운서
지오스톰
이마트 트레이더스 휴무일
로또867회당첨번호
원더풀 고스트
선우선
보좌관 시즌2
호텔 델루나
이수민
안미나
원피스 893화 애니
델루나
인기가요
미구하라
트와이스 미나
제니하우스 염색약
우하람
홍윤화
있지
아스달 연대기 방송시간
정세진 아나운서
지오스톰
이마트 트레이더스 휴무일


In [21]:
#list에 담아서 겹치는거 제거

keywords = []

for keyword in elements:
    keywords.append(keyword.find("span",{"class":"ah_k"}).find(text=True))
    
keywords = keywords[0:20]


In [36]:
for rank in elements:
    print(rank.find("span",{"class":"ah_r"}))

    
points = []

for point in elements:
    points.append(point.find("span",{"class":"ah_r"}).find(text=True))
    



<span class="ah_r">1</span>
<span class="ah_r">2</span>
<span class="ah_r">3</span>
<span class="ah_r">4</span>
<span class="ah_r">5</span>
<span class="ah_r">6</span>
<span class="ah_r">7</span>
<span class="ah_r">8</span>
<span class="ah_r">9</span>
<span class="ah_r">10</span>
<span class="ah_r">11</span>
<span class="ah_r">12</span>
<span class="ah_r">13</span>
<span class="ah_r">14</span>
<span class="ah_r">15</span>
<span class="ah_r">16</span>
<span class="ah_r">17</span>
<span class="ah_r">18</span>
<span class="ah_r">19</span>
<span class="ah_r">20</span>
<span class="ah_r">1</span>
<span class="ah_r">2</span>
<span class="ah_r">3</span>
<span class="ah_r">4</span>
<span class="ah_r">5</span>
<span class="ah_r">6</span>
<span class="ah_r">7</span>
<span class="ah_r">8</span>
<span class="ah_r">9</span>
<span class="ah_r">10</span>
<span class="ah_r">11</span>
<span class="ah_r">12</span>
<span class="ah_r">13</span>
<span class="ah_r">14</span>
<span class="ah_r">15</span>
<sp

In [39]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np

df({"실시간키워드":keywords, "순위":point[:20]})


TypeError: unhashable type: 'slice'

# 4. 네이버 포스트 크롤링

아래는 고려대학교 김보섭님의 자료를 기반으로 만들어졌습니다.

In [10]:


response = requests.get('https://search.naver.com/search.naver?where=post&sm=tab_pge&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC&st=sim&date_option=0&date_from=&date_to=&dup_remove=1&post_blogurl=&post_blogurl_without=&srchby=all&nso=&ie=utf8&start=1')
bs = BeautifulSoup(response.text, 'html.parser')

In [11]:
contents = bs.select('ul#elThumbnailResultArea li.sh_blog_top dt')

In [12]:
contents[0:3]

[<dt><a class="sh_blog_title _sp_each_url _sp_each_title" href="https://blog.naver.com/since201109?Redirect=Log&amp;logNo=221553965961" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=1&amp;i=90000003_000000000000003395A53B89&amp;u='+urlencode(this.href))" target="_blank" title="한권으로 끝내는 파이썬 - 독학으로 Python 프로그램 코딩 공부하기">한권으로 끝내는 <strong class="hl">파이썬</strong> - 독학으로 Python 프로그램 코딩...</a></dt>,
 <dt><a class="sh_blog_title _sp_each_url _sp_each_title" href="https://blog.naver.com/stochastic73?Redirect=Log&amp;logNo=221574101454" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=2&amp;i=90000003_000000000000003396D879CE&amp;u='+urlencode(this.href))" target="_blank" title="금융권에 필요한 파이썬 스킬트리 #1.">금융권에 필요한 <strong class="hl">파이썬</strong> 스킬트리 #1.</a></dt>,
 <dt><a class="sh_blog_title _sp_each_url _sp_each_title" href="https://blog.naver.com/elepartsblog?Redirect=Log&amp;logNo=221583231746" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=3&amp;i=90000003_00000000000000339763CB02&amp

In [13]:
#블로그 포스트 제목

contents[0].select_one('a').attrs.get('title')

'한권으로 끝내는 파이썬 - 독학으로 Python 프로그램 코딩 공부하기'

In [14]:
#블로그 포스트 주소

contents[0].select_one('a').attrs.get('href')

'https://blog.naver.com/since201109?Redirect=Log&logNo=221553965961'

In [15]:
data = [
    {'title': content.select_one('a').attrs.get('title'),
    'address' : content.select_one('a').attrs.get('href')}
    for content
    in contents
]

In [16]:
pd.DataFrame(data)

address  \
0  https://blog.naver.com/since201109?Redirect=Lo...   
1  https://blog.naver.com/stochastic73?Redirect=L...   
2  https://blog.naver.com/elepartsblog?Redirect=L...   
3          http://blog.fastcampus.co.kr/221531192167   
4              https://cosmosjs.blog.me/221521309659   
5  https://blog.naver.com/jjys9047?Redirect=Log&l...   
6  https://blog.naver.com/infopub?Redirect=Log&lo...   
7          https://smartplus777.blog.me/221571925511   
8  https://blog.naver.com/micrmm?Redirect=Log&log...   
9  https://blog.naver.com/hjchoi1500?Redirect=Log...   

                                               title  
0            한권으로 끝내는 파이썬 - 독학으로 Python 프로그램 코딩 공부하기  
1                              금융권에 필요한 파이썬 스킬트리 #1.  
2               라즈베리파이에서 파이썬으로 1602 I2C 캐릭터 LCD 사용하기  
3  프로그래밍의 세계를 여행하는 프.알.못을 위한 안내서 - #2.파이썬 왜 쉽다고 하나요?  
4  라즈베리파이(파이썬)에서 Firebase의 Firestore Database 이용하...  
5                         파이썬 기초 [금통위의사록 파이썬으로 다운받기]  
6                파이썬 코딩 수학 with 딥러닝 : 수학적 사고력 향상을 위한  
7                                        파이썬으로 게임만들기  
8                        파이썬 독학 입문 책 추천, 혼자 공부하는 파이썬  
9                         파이썬 머신러닝, 연관성 분석 (+ 학원 정보)

과제 

----------------------필수 ------------------------

1. "http://pythonhow.com/example.html" 에서 도시명&도시설명 크롤링

2. "http://www.pythonscraping.com/pages/warandpeace.html" 에서 이름&대사 대사 크롤링

----------------------둘 중 하나 선택 ---------------------------------

3. 원하는 영화 리뷰(제목, 내용) 크롤링하기 100개 이상
   ex)https://movie.naver.com/movie/bi/mi/review.nhn?code=168298&page=1

4. TripAdviser 서울 호텔 리뷰 크롤링 5 페이지 이상
  -더 보기 전까지만 크롤링
  //(BeautifulSoup을 활용하여 "더 보기"를 클릭하여 전체 리뷰 보는 것 !! "Chalenge"

# 텍스트 마이닝 

# 1. 

In [6]:
#페이지 넘어가면서 자동화하기 50페이지까지 긁어보기

import re

url_list = []

for i in range(51):
    url = "https://movie.naver.com/movie/point/af/list.nhn?&page="+str(i)
    url_list.append(url)
    
url_list[1:51]
points = []
titles = []
contents = []

for url in url_list[1:30]:
    #평점

    html = urlopen(url)
    soup = BeautifulSoup(html,"html.parser",from_encoding='utf-8')

    reviews = soup.findAll("tr")
    
    for item in reviews[1:]:
        titles.append(item.find("a",{"class":"movie"}).find(text=True))
        points.append(item.find("td",{"class":"point"}).find(text=True))
        contents.append(item.find("a",{"class":"report"})["href"])
        
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') 

contents_hangel = []
for content in contents:
    contents_hangel.append(hangul.sub('', content))
    
contents_hangel

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


['  감정을 들었다 놓았다 하는 음악  ',
 '  소더버그 감독은 하이스트 무비의 명장 ㅎ 유쾌 발랄 유머와 감동까지 전하는 신나는 영화   ',
 '  제발 씹덕들은 영화관 오지마라 좇같다  ',
 '  편보단 약간 성이 안차지만 그래도 두눈이 호강했네 ㅋㅋㅋ  ',
 '  미국은 참 멋있는 나라다  ',
 '  이딴 냄새나는 영화가 점 ㅁㅊ냐  ',
 '  명작이네요 꼭 보세요  ',
 '  요리들땜에  배고팠음  ',
 '  오터쿠는 나가 죽어야 돼  ',
 '  간만에  즐겁게 본 스릴러  머리쓰는 재미도 있고 김강우 슈트빨  ㄷㄷ  ',
 '  파오후 쿰척쿰척 파오후 쿰척쿰척  ',
 '  파오후 쿰척쿰척 킁킁킁  ',
 '  시간가는 줄 모르고 봄 특히 후반부 분은 ㅗㅜㅑ  ',
 '  관상에 대해 실망을 강하게 주어 풍수의 재미를 한껏 올리려는 속셈이렸다 성공했구나 성공했어 아주 큰 성공을 하였구나  ',
 '  스케일이 전작 다 씹어먹음 다른 영화 스케일은 눈에 안찰듯  ',
 '  늦었는데 예거타고 출근하고 싶다  ',
 '  지랄똥싸지말고 주차공짜로 잘하고간다  ',
 '  파오후들은 눈치안보고 이런거잘보러오더라  ',
 '  파오후 놈들아 너네들은 영화관 좀 오지마 땀냄새나서 다른 사람들 다 짜증나  ',
 '  명작이니 뭐니 헛소리하길래 점 드립니다  ',
 '  다시는 영화판에 들어오지 말아라  ',
 '  보다 재미짐 스케일을 다 담기에 내가 본 스크린이 너무 작았음 주말에 아맥으로 한번 더 봐야지  ',
 '  싫어하는 데에 별 이유는 없다  ',
 '  ㅡㅡ 별테두리도 아까워  ',
 '  파오후 쿰척쿰척 잠재적 아동성범죄자 오타쿠새기덜  ',
 '  무조건 아이맥스로 봐야 진리ㄱㄱ 스케일 오지구요 지리구요  ',
 '  디지고싶냐 오타쿠새기들아  ',
 '  누군가 날 기억해 준다는게 얼마나 가슴벅차는 감동인지 알게 된 영화네요 어른들이 더 감동할 영화인듯 싶습니다   ',
 '  리틀 포레스트 보고왔어요영화에서 주고자하는 핵

In [7]:
contents_hangel

['  감정을 들었다 놓았다 하는 음악  ',
 '  소더버그 감독은 하이스트 무비의 명장 ㅎ 유쾌 발랄 유머와 감동까지 전하는 신나는 영화   ',
 '  제발 씹덕들은 영화관 오지마라 좇같다  ',
 '  편보단 약간 성이 안차지만 그래도 두눈이 호강했네 ㅋㅋㅋ  ',
 '  미국은 참 멋있는 나라다  ',
 '  이딴 냄새나는 영화가 점 ㅁㅊ냐  ',
 '  명작이네요 꼭 보세요  ',
 '  요리들땜에  배고팠음  ',
 '  오터쿠는 나가 죽어야 돼  ',
 '  간만에  즐겁게 본 스릴러  머리쓰는 재미도 있고 김강우 슈트빨  ㄷㄷ  ',
 '  파오후 쿰척쿰척 파오후 쿰척쿰척  ',
 '  파오후 쿰척쿰척 킁킁킁  ',
 '  시간가는 줄 모르고 봄 특히 후반부 분은 ㅗㅜㅑ  ',
 '  관상에 대해 실망을 강하게 주어 풍수의 재미를 한껏 올리려는 속셈이렸다 성공했구나 성공했어 아주 큰 성공을 하였구나  ',
 '  스케일이 전작 다 씹어먹음 다른 영화 스케일은 눈에 안찰듯  ',
 '  늦었는데 예거타고 출근하고 싶다  ',
 '  지랄똥싸지말고 주차공짜로 잘하고간다  ',
 '  파오후들은 눈치안보고 이런거잘보러오더라  ',
 '  파오후 놈들아 너네들은 영화관 좀 오지마 땀냄새나서 다른 사람들 다 짜증나  ',
 '  명작이니 뭐니 헛소리하길래 점 드립니다  ',
 '  다시는 영화판에 들어오지 말아라  ',
 '  보다 재미짐 스케일을 다 담기에 내가 본 스크린이 너무 작았음 주말에 아맥으로 한번 더 봐야지  ',
 '  싫어하는 데에 별 이유는 없다  ',
 '  ㅡㅡ 별테두리도 아까워  ',
 '  파오후 쿰척쿰척 잠재적 아동성범죄자 오타쿠새기덜  ',
 '  무조건 아이맥스로 봐야 진리ㄱㄱ 스케일 오지구요 지리구요  ',
 '  디지고싶냐 오타쿠새기들아  ',
 '  누군가 날 기억해 준다는게 얼마나 가슴벅차는 감동인지 알게 된 영화네요 어른들이 더 감동할 영화인듯 싶습니다   ',
 '  리틀 포레스트 보고왔어요영화에서 주고자하는 핵

#### from konlpy.corpus import kolaw
from konlpy.tag import Twitter

twitter = Twitter()
tagged_text = [twitter.pos(line) for line in contents_hangel]
tagged_text

In [11]:
import nltk
parser_ko = nltk.RegexpParser("NP: {<Adjective>*<Noun>*}")
chunks_ko = parser_ko.parse(tagged_text[1])
chunks_ko.draw()

In [37]:
tmp_text = []
for line in tagged_text:
        tmp_text.append([morph[0] for morph in line if morph[1].find('Noun') >=2 or morph[1].find('Verb')>=2 
                        or morph[1].find('Adjective')])
final_text = [' '.join(tmp) for tmp in tmp_text]


final_text

['감정 을 들었 다 놓 았 다 하는 음악',
 '소더 버그 감독 은 하 이스트 무비 의 명장 ㅎ 유쾌 발랄 유머 와 감동 까지 전하는 신 나 는 영화',
 '제발 씹덕 들 은 영화관 오지 좇같 다',
 '편 보단 약간 성 이 안 차지 만 그래도 두 눈 이 호강 했 네 ㅋㅋㅋ',
 '미국 은 참 나라 다',
 '이 딴 냄새 나 는 영화 가 점 ㅁㅊ 냐',
 '명작 이네 요 꼭 보세 요',
 '요리 들 땜 에 음',
 '오터쿠 는 나가 죽어 야 돼',
 '간만 에 게 본 스릴러 머리 쓰는 재미 도 김강우 슈트 빨 ㄷㄷ',
 '파오 후 쿰척쿰척 파오 후 쿰척쿰척',
 '파오 후 쿰척쿰척 킁킁킁',
 '시간 가는 줄 모르 고 봄 특히 후반 부 분 은 ㅗㅜㅑ',
 '관상 에 대해 실망 을 주어 풍수 의 재미 를 한껏 올리 려는 속셈 이 렸 다 성공했 구나 성공했 어 아주 큰 성공 을 하였 구나',
 '스케일 이 전작 다 씹어 먹음 다른 영화 스케일 은 눈 에 안 찰 듯',
 '늦었 는데 예거 타고 출근하고 싶다',
 '지랄 똥 싸지 말고 주차 공짜 로 잘하고 간다',
 '파오 후 들 은 눈치 안보 고 거 잘 보러 오 더라',
 '파오 후 놈 들 아 너 네 들 은 영화관 좀 오지 마 땀 냄새 나서 다른 사람 들 다 짜증 나',
 '명작 이니 뭐 니 헛소리 하길 래 점 드립니 다',
 '다시 는 영화 판 에 들어오지 말 아라',
 '보다 재미 짐 스케일 을 다 담기 에 내 가 본 스크린 이 너무 음 주말 에 아 맥 으로 한번 더 봐 야지',
 '데 에 별 이유 는',
 'ㅡㅡ 별 테두리 도',
 '파오 후 쿰척쿰척 잠재 적 아동 성범죄자 오타쿠 새기 덜',
 '무조건 아이맥스 로 봐 야 진리 ㄱㄱ 스케일 오지 구요 지리 구요',
 '디지 고 싶 냐 오타쿠 새기 들 아',
 '누군가 날 기억해 준 다 는 게 얼마나 가슴 벅 차는 감동 인지 알 게 된 영화 네 요 어른 들 이 더 감동 할 영화인 듯 싶 습니다',
 '리틀 포레스트 보고 왔 어요 영화 에서 주고 자하 는 핵심

빈 리스트와 같이 필요없는 것들이 들어있을 경우 이를 제거하기 위해 사용할 뿐 필수 코드는 아니다

In [45]:
#중복요소 제거

ex_list = list(set(final_text))

ex_list

['',
 '아따 여기 여 좌표 가 맞 능가',
 '파오 후 는 죽은 파오 후 다 거기 에 오타쿠 일 경우 진심 자살 을 추천한 다',
 '제발 씹덕 들 은 영화관 오지 좇같 다',
 '개 쓰레기 영화 ㅋㅋㅋ 내용 도 쓰레기 배우 들 연기 도 쓰레기 임창정 은 색즉시공 때 랑 지금 이랑 달라진 게 아무래도 감독 이 약 을 쳐 빨 고 캐스팅 했 나 보다 정상훈 이 주연 이라니 ㅋㅋ',
 '아직 점 안준 흑우 제',
 '이 페 도 새끼 들 거 보면 서 자괴 감 안 드냐 ㅡㅡ',
 '점 되는 영화 는 라고 개인 적 으로 생각합 니다 쇼핑몰 에서도 고 밖 에서도 고 허리케인 으로 인해 모든 날아다니는 파편 물 들 이 악당 들 만 맞고 주인공 들 은 몸 에 상처 하나 총상 하나 안 입는 게 너무 몰입 감 을 떨어뜨립니 다',
 '상영 관 게 것 좀 수입하지',
 '파오 후 쿰척쿰척 킁킁킁',
 '보는 내 내 고 고 고 뭉클했 다 ㅠ',
 '에이 우리 아들 이 리가 욧 애니메이션 볼 나이 는 지났 다 구요',
 '진짜 최 아 쓰 렉 영화 다',
 '급 을 넘어선 급 영화 다 병맛 을 뛰어넘 었 습니다',
 '델 토로 가 퍼시픽 림 붕어 붕어빵 처럼 의미',
 '간만 에 게 본 스릴러 머리 쓰는 재미 도 김강우 슈트 빨 ㄷㄷ',
 '파오 후 쿰척쿰척',
 '덕 덕우 웩 다',
 '생각 이 새긴 가 국제 적 으로 개 망신 시킬 일',
 '파오 후 놈 들 아 너 네 들 은 영화관 좀 오지 마 땀 냄새 나서 다른 사람 들 다 짜증 나',
 '오타쿠 우웩',
 '야붕이들 아 야야 야 는 솔 찍히 인정해 줘 야 하지 않냐 점주 는 쉐리 들 코갤러 라 보면 되는 부분 이 냐',
 '오타쿠 들 또 설치 네 ㅋㅋ',
 '오타쿠 만화 네 요 극혐 이에요 스크린 에 것 점 걸지 마 요 영화관 가면 거 보는 사람 몸 에서 냄새 나 요',
 '명작 이니 뭐 니 헛소리 하길 래 점 드립니 다',
 '생생 한 감동 이 밀려 와 서 맘 이 짠 했 습니다',
 '파오 후 쿰척쿰척 파오 후 쿰척쿰척',

# DocumentTermMatrix with tf

In [54]:
import os, sys
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [22]:
# 3글자 이상의 영어단어만 각 문서에서 추출
corpus = ['//'.join(re.findall('[ㄱ-ㅣ가-힣]{3,}', doc)) for doc in ex_list]

corpus

NameError: name 'ex_list' is not defined

In [52]:
def string_tokenizer(doc):
    return doc.split('//')

In [57]:
# tokenizer option에 내가 정의한 tokenizer를 명시
# min_df로 최소 토큰의 빈도 설정 (Term의 list를 먼저 만들어두지 않았을 경우 활용할만한 옵션)
tf_dtm = CountVectorizer(tokenizer = string_tokenizer, min_df = 1).fit(corpus) 

tf_dtm

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function string_tokenizer at 0x0000022A62338598>,
        vocabulary=None)

In [61]:
tf_dtm.transform(corpus).toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# DocumentTermMatrix with tf-idf

In [62]:
tfidf_dtm= TfidfVectorizer(tokenizer = string_tokenizer).fit(corpus)

In [66]:
tfidf_dtm.transform(corpus).toarray()

C:\Users\glam\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

다른 예시 : 이런 형태입니다!!

array([[ 0.        ,  0.47936124,  0.        ,  0.        ,  0.47936124,
         0.        ,  0.23968062,  0.23968062,  0.        ,  0.23968062,
         0.        ,  0.23968062,  0.23968062,  0.23968062,  0.23968062,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.15298503,  0.23968062,  0.23968062,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.36742339,  0.        ,  0.        ,
         0.36742339,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.36742339,  0.        ,  0.36742339,  0.        ,  0.        ,
         0.        ,  0.36742339,  0.23452159,  0.        ,  0.        ,
         0.36742339,  0.36742339,  0.        ],
       [ 0.33333333,  0.        ,  0.        ,  0.33333333,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.66666667,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.33333333,  0.        ,
         0.33333333,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.33333333],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.5417361 ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.5417361 ,  0.        ,  0.        ,  0.5417361 ,
         0.        ,  0.        ,  0.34578314,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ]])

# word2vec

1. 100차원의 벡터로 embedding
2. 초기 learning rate = 0.025
3. window size = 5
4. min_count = 2 (최소 2회이상 나타난 단어만)
5. skip-gram

In [17]:
from gensim.models.word2vec import Word2Vec
token = [s.split() for s in ex_list]
config = {'size' :100, 'alpha' : 0.025, 'window' : 100, 'sg' : 1, 'min_count' : 2} 
# sg는 skip-gram 방법의 사용여부요 1이면 skip-gram, 0이면 CBOW
model = Word2Vec(sentences = token, **config)


C:\Users\glam\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [22]:
# 유사도 검증을 위해 token들의 집합을 하나 따로 떼어두기

from collections import Counter
tokens = sum(token, [])
tokens = Counter(tokens)
tokens = [token[0] for token in list(tokens.items()) if token[1] >= 2]

tokens

['급',
 '었',
 '드디어',
 '가요',
 '구',
 '갑',
 '퍼시픽림',
 '때문',
 '싶',
 '날',
 '과',
 '해',
 '눈물',
 '덜',
 '알바',
 '점주',
 '대박',
 '흠',
 '냐',
 '요',
 '간만',
 '들',
 '주',
 '그냥',
 '한',
 '않',
 '지',
 '야',
 '쪽',
 '감정',
 '눈',
 '큰',
 '풍기',
 '구나',
 '맛',
 '반전',
 '밖에',
 '죽은',
 '자체',
 '타',
 'ㅠ',
 '운',
 '디지',
 '였',
 '쿰척',
 '개꿀잼',
 '관',
 '물론',
 '과는',
 '내용',
 '뜨',
 '얼굴',
 '진짜',
 '씹',
 '무슨',
 '준',
 '쓰레기',
 '맥',
 '모르',
 '랑',
 '이건',
 '뿐',
 '보여',
 '오늘',
 '너',
 '기대',
 '너무',
 '안',
 '땀',
 '다',
 '용',
 '내일',
 '돈',
 '만드는',
 '조조',
 '새끼',
 '후',
 '줄',
 '시',
 '본',
 '별',
 '왜',
 '마지막',
 '된',
 '골든',
 '아저씨',
 '모든',
 '드립니',
 '카',
 '듯',
 '스크린',
 '쉐리',
 '와',
 '로봇',
 '거리',
 '완전',
 '작품',
 '십',
 '것',
 '되',
 '개봉',
 '아들',
 '점',
 '웩',
 '쿠',
 '은',
 '억',
 '오타쿠',
 '레즈비언',
 '잘',
 '상영',
 '간다',
 '나이',
 '겁나',
 '좌표',
 '버러지',
 '할',
 '풀어',
 '던',
 '함',
 '사랑',
 '이니',
 '의',
 '편',
 '일본',
 '임창정',
 '에',
 '마',
 '받았',
 '극장판',
 '영화관',
 '씹네',
 '버',
 '지도',
 '보면',
 '대',
 '지지',
 '중',
 '보는',
 '면서',
 '겠',
 '를',
 '소녀',
 '자',
 '니다',
 '척척',
 '애니',
 '도',
 '극장',
 '놈',
 '다음',

In [ ]:
from gensim.models.word2vec import Word2Vec
token = [s.split() for s in ex_list]
config = {'size' :100, 'alpha' : 0.025, 'window' : 100, 'sg' : 1, 'min_count' : 2} 
# sg는 skip-gram 방법의 사용여부요 1이면 skip-gram, 0이면 CBOW
model = Word2Vec(sentences = token, **config)


In [23]:
model.init_sims(replace = True) #필요없는 메모리 unload

In [33]:
# 두 word간의 embedding 공간상의 유사도 계산
model.similarity(tokens[3], tokens[5])


0.9989317425240217

In [34]:
model.wv["쓰레기"]

array([-0.13632241, -0.0513466 ,  0.12980488,  0.01153469, -0.10995204,
       -0.06064469, -0.08759674,  0.05343053, -0.28631058,  0.00452097,
       -0.12488635, -0.12649171,  0.02441184,  0.02305884,  0.02281335,
       -0.00384885,  0.22098778,  0.09204169, -0.04975539, -0.019643  ,
       -0.05666046,  0.0479789 ,  0.01424258, -0.07599976, -0.0054203 ,
        0.02547973, -0.00162094, -0.01956542, -0.3413037 , -0.09955025,
       -0.14117403, -0.16476439, -0.10581765, -0.0156876 ,  0.15418003,
       -0.09424199,  0.04585955, -0.03611828,  0.02299822, -0.10364172,
        0.00268552,  0.22092983,  0.07145649, -0.01834189,  0.05001862,
        0.02122566, -0.15510847,  0.02719334, -0.02743221,  0.07688574,
       -0.18020801,  0.02555726,  0.02972159,  0.08733013,  0.02191799,
       -0.08901552, -0.13551131, -0.01110751,  0.02342053, -0.04972407,
        0.0639587 , -0.0886104 , -0.04880271, -0.10294965, -0.10081596,
       -0.0321299 ,  0.01145669,  0.05771445, -0.12525244,  0.14

In [35]:
model.most_similar("쓰레기")

[('고', 0.9995486736297607),
 ('이', 0.9995008111000061),
 ('도', 0.9994970560073853),
 ('이랑', 0.9994908571243286),
 ('은', 0.99949049949646),
 ('이라니', 0.9994885921478271),
 ('했', 0.9994871616363525),
 ('랑', 0.999487042427063),
 ('엄청', 0.9994771480560303),
 ('게', 0.9994763135910034)]

In [38]:
# 단어간 관계 찾기
model.most_similar(positive = ['쓰레기', '엄청'], negative = ['얼굴'])

[('은', 0.9987679123878479),
 ('한마디', 0.9987372756004333),
 ('때', 0.998694658279419),
 ('랑', 0.9986813068389893),
 ('돼지', 0.9986779689788818),
 ('도', 0.9986656308174133),
 ('웃고', 0.9986638426780701),
 ('이랑', 0.9986629486083984),
 ('사람', 0.9986605644226074),
 ('한', 0.9986361861228943)]

In [474]:
import numpy as np
my_word = model.wv.index2word

my_word

['이',
 '영화',
 '다',
 '는',
 '의',
 '가',
 '은',
 '고',
 '들',
 '을',
 '에',
 '를',
 '도',
 '서',
 '한',
 '너무',
 '요',
 '게',
 '나',
 '음',
 '지',
 '봤',
 '만',
 '어요',
 '것',
 '보고',
 '적',
 '로',
 '수',
 '거',
 '원작',
 '사람',
 '진짜',
 '과',
 '안',
 '으로',
 '잘',
 '시간',
 '니다',
 '연기',
 '는데',
 '더',
 '평점',
 '습니다',
 '그',
 '했',
 '일본',
 '았',
 '데',
 '듯',
 '내',
 '였',
 '때',
 '에서',
 '인',
 '배우',
 '주',
 '생각',
 '싶',
 '야',
 '던',
 '면',
 '하는',
 '보는',
 '봐',
 '와',
 '손예진',
 '이다',
 '라',
 '감동',
 '하고',
 '점',
 '보다',
 '장면',
 '까지',
 '네요',
 '내용',
 '네',
 '정말',
 '한국',
 '어',
 '좀',
 '편',
 '못',
 '돈',
 '급',
 '눈물',
 '하',
 '볼',
 '아',
 '느낌',
 '화',
 '본',
 '성',
 '명작',
 'ㅠㅠ',
 '감독',
 '그냥',
 '스토리',
 'ㅋㅋㅋ',
 '되는',
 '하나',
 '만들어',
 '사랑',
 '인생',
 '마지막',
 '웹툰',
 '넘',
 '많이',
 '지만',
 '에게',
 '말',
 '한국판',
 '할',
 '재미',
 '작품',
 '가슴',
 '짜',
 '있',
 '줄',
 '길',
 '뭐',
 '저',
 '다시',
 '최악',
 '겠',
 '유치',
 '한국영',
 '반전',
 '이었',
 '라는',
 '전개',
 '감정',
 '개',
 '하다',
 '이고',
 '않았',
 '꼭',
 '웃음',
 '대사',
 '같이',
 '아이',
 '판',
 '자체',
 '시',
 '중간',
 'ㅋㅋ',
 '현실',
 '정도',
 '었',
 '뭔가',
 '만든',
 '애'

In [39]:
# 모든 word들의 embedding matrix 생성
import numpy as np
my_word = model.wv.index2word

In [40]:
embedding = [model[token] for token in my_word]
embedding[0:2]

[array([-0.13299738, -0.05382725,  0.13195157,  0.01300956, -0.1102557 ,
        -0.059519  , -0.08789597,  0.0576382 , -0.2884419 ,  0.00347979,
        -0.12427967, -0.12633747,  0.02190861,  0.02183352,  0.02546261,
         0.00271465,  0.21932693,  0.09108165, -0.04760861, -0.0233534 ,
        -0.05790456,  0.04303058,  0.01735174, -0.07850829,  0.00036262,
         0.02368691, -0.00365983, -0.02008701, -0.34357697, -0.10043832,
        -0.13536137, -0.16558585, -0.11164851, -0.01653728,  0.15242174,
        -0.09815661,  0.04554345, -0.03608039,  0.02289013, -0.10273112,
        -0.00275003,  0.21759062,  0.06744189, -0.01451005,  0.04768943,
         0.02285046, -0.15993881,  0.02407633, -0.02696448,  0.07102775,
        -0.18164559,  0.02858835,  0.02555747,  0.08761804,  0.0252762 ,
        -0.0902189 , -0.12893413, -0.01412691,  0.02140577, -0.05489014,
         0.06225543, -0.08933569, -0.05343965, -0.1024564 , -0.09831785,
        -0.0378024 ,  0.0155087 ,  0.05240559, -0.1

In [41]:
embedding = np.asarray(embedding)
embedding

array([[-0.13299738, -0.05382725,  0.13195157, ..., -0.13942531,
        -0.03320158, -0.05918336],
       [-0.12763768, -0.05531897,  0.13445431, ..., -0.13612054,
        -0.03735719, -0.0632589 ],
       [-0.12700482, -0.05123206,  0.13431919, ..., -0.14156789,
        -0.03369409, -0.05758893],
       ...,
       [-0.12961304, -0.05659704,  0.13435017, ..., -0.13390774,
        -0.03429692, -0.0553656 ],
       [-0.12945516, -0.05452174,  0.13392858, ..., -0.13608009,
        -0.03986555, -0.05950816],
       [-0.1274952 , -0.05912019,  0.13945921, ..., -0.1346066 ,
        -0.04202734, -0.05554883]], dtype=float32)

In [475]:
model.wv.save_word2vec_format('GG.txt', binary=False)

In [476]:
from pylab import figure, axes, pie, title, savefig

def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    fig = plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()
    fig.savefig('GG.jpg')

# Doc2vec

문서(document), 문단(paragraph), 또는 문장(sentence)를 Continuous vector로 표현하는 방법인 doc2vec(paragraph vector)를 해봅니다.


In [67]:
#load modules

import os, sys
import nltk
import re
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
os.listdir()

['.gitignore',
 '.httr-oauth',
 '.ipynb_checkpoints',
 '.RData',
 '.RDataTmp',
 '.RDataTmp1',
 '.RDataTmp2',
 '.RDataTmp3',
 '.Rhistory',
 '0_tf_basic.ipynb',
 '2018 뷰솝 - 텍마 강의.ipynb',
 '2018 텍마 강의.ipynb',
 '5_CNN',
 'aclImdb_v1',
 'assignment1',
 'b.jpg',
 'best_model.ckpt.data-00000-of-00001',
 'best_model.ckpt.index',
 'best_model.ckpt.meta',
 'bigcon.ipynb',
 'bigcon.zip',
 'cacert.pem',
 'cc.txt',
 'checkpoint',
 'CNN classifier.ipynb',
 'CNN.ipynb',
 'code',
 'darktower.txt',
 'desktop.ini',
 'DNN HOMEWORK.ipynb',
 'Documents (2).zip',
 'Documents (3).zip',
 'Documents.zip',
 'embedding1.csv',
 'ezPDFReader3.0',
 'gensim-2.1.0-cp35-cp35m-win_amd64',
 'gensim-2.1.0-cp35-cp35m-win_amd64.whl',
 'GG.csv',
 'GG.jpg',
 'GG.pdf',
 'GG.txt',
 'GIS DataBase',
 'gun.jpg',
 'gunhamdo.txt',
 'hotel_reviews_50.cell',
 'hotel_reviews_50und_rate_1_1.csv',
 'hotel_word2vec.txt',
 'images',
 'int_summaries.txt',
 'int_texts.txt',
 'int_to_vocab.txt',
 'jangsan.txt',
 'Jongseob_love.csv',
 'JPype1

In [85]:

token = [s.split() for s in ex_list]

token

[[],
 ['아따', '여기', '여', '좌표', '가', '맞', '능가'],
 ['파오',
  '후',
  '는',
  '죽은',
  '파오',
  '후',
  '다',
  '거기',
  '에',
  '오타쿠',
  '일',
  '경우',
  '진심',
  '자살',
  '을',
  '추천한',
  '다'],
 ['제발', '씹덕', '들', '은', '영화관', '오지', '좇같', '다'],
 ['개',
  '쓰레기',
  '영화',
  'ㅋㅋㅋ',
  '내용',
  '도',
  '쓰레기',
  '배우',
  '들',
  '연기',
  '도',
  '쓰레기',
  '임창정',
  '은',
  '색즉시공',
  '때',
  '랑',
  '지금',
  '이랑',
  '달라진',
  '게',
  '아무래도',
  '감독',
  '이',
  '약',
  '을',
  '쳐',
  '빨',
  '고',
  '캐스팅',
  '했',
  '나',
  '보다',
  '정상훈',
  '이',
  '주연',
  '이라니',
  'ㅋㅋ'],
 ['아직', '점', '안준', '흑우', '제'],
 ['이', '페', '도', '새끼', '들', '거', '보면', '서', '자괴', '감', '안', '드냐', 'ㅡㅡ'],
 ['점',
  '되는',
  '영화',
  '는',
  '라고',
  '개인',
  '적',
  '으로',
  '생각합',
  '니다',
  '쇼핑몰',
  '에서도',
  '고',
  '밖',
  '에서도',
  '고',
  '허리케인',
  '으로',
  '인해',
  '모든',
  '날아다니는',
  '파편',
  '물',
  '들',
  '이',
  '악당',
  '들',
  '만',
  '맞고',
  '주인공',
  '들',
  '은',
  '몸',
  '에',
  '상처',
  '하나',
  '총상',
  '하나',
  '안',
  '입는',
  '게',
  '너무',
  '몰입',
  '감',
  '을',
  '떨어뜨립니',
  '다']

gensim의 Doc2Vec class가 input으로 받을 수 있는 corpus 형태로 변환
(collections module의 namedtuple 이용)

In [86]:

from collections import namedtuple
TaggedDocument = namedtuple('TaggedDocument', ['words', 'tags'])

tagged_train_docs = [TaggedDocument(words, [tags]) for tags, words in enumerate(token)]
tagged_train_docs[1:5]

[TaggedDocument(words=['아따', '여기', '여', '좌표', '가', '맞', '능가'], tags=[1]),
 TaggedDocument(words=['파오', '후', '는', '죽은', '파오', '후', '다', '거기', '에', '오타쿠', '일', '경우', '진심', '자살', '을', '추천한', '다'], tags=[2]),
 TaggedDocument(words=['제발', '씹덕', '들', '은', '영화관', '오지', '좇같', '다'], tags=[3]),
 TaggedDocument(words=['개', '쓰레기', '영화', 'ㅋㅋㅋ', '내용', '도', '쓰레기', '배우', '들', '연기', '도', '쓰레기', '임창정', '은', '색즉시공', '때', '랑', '지금', '이랑', '달라진', '게', '아무래도', '감독', '이', '약', '을', '쳐', '빨', '고', '캐스팅', '했', '나', '보다', '정상훈', '이', '주연', '이라니', 'ㅋㅋ'], tags=[4])]

# Training doc2vec

1. size = 100 : 100차원의 벡터로 embedding (document와 word 모두, 이 때의 word embedding이 word2vec과 같은 효과를 가지는 지는 검증이 필요)

2. dm = 1: pv-dm (paragraph vector : distributed memory)

3. dm_concat = 1 : 1-layer의 hidden node의 값들을 구성할 때, word vector와 paragraph vector concatenate 함

4. min_count = 2 : 최소 2회이상 나타난 단어만

In [100]:
from gensim.models import doc2vec
# 사전 구축
config = {'size' : 100, 'dm_concat' : 1, 'dm' : 1, 'min_count' : 1}
doc_vectorizer = doc2vec.Doc2Vec(**config)

In [101]:
doc_vectorizer.build_vocab(tagged_train_docs)

In [103]:
# Train document vectors!

for epoch in range(100):
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002  # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

In [104]:
my_word = doc_vectorizer.wv.index2word
word_embedding = [doc_vectorizer.wv[token] for token in my_word]
word_embedding = np.asarray(word_embedding)

np.asarray(doc_vectorizer.docvecs).shape

(218, 100)

In [105]:
word_embedding

array([[ 0.5926128 ,  0.45262256, -0.11925705, ...,  0.2739611 ,
         0.791628  , -0.06056391],
       [ 0.3414498 ,  0.30422968, -0.01862387, ...,  0.40196607,
         0.48387316, -0.10012004],
       [ 0.88379675,  0.59167093, -0.0255428 , ...,  0.19527899,
         1.0105442 , -0.08575758],
       ...,
       [ 0.15542336,  0.21167785, -0.02915411, ...,  0.16050386,
         0.27684224, -0.02306661],
       [ 0.12955746,  0.16732146,  0.02343283, ...,  0.21271299,
         0.22613533, -0.02871538],
       [-0.07274573,  1.7597079 ,  0.9935873 , ...,  4.0039873 ,
         1.6502662 , -0.44501045]], dtype=float32)

비정형 데이터를 우리가 원하는 벡터 형태로 표현했으니

분류, 요약, 의미 분석, 감성 분석 등이 가능하겠죠!?

